# Libraries

In [ ]:
import pandas as pd

# External Data

## Barrios

In [ ]:
df_barrios = pd.read_csv("../data/external/barrios/barrios_caba.csv")
for i in df_barrios["barrio"].unique():
    if i.lower() not in all_neigborhoods:
        all_neigborhoods.append(i.strip().lower())

## Transporte

In [ ]:
df_transport = pd.read_parquet("../data/processed/transporte/transporte.parquet", engine="pyarrow")
df_transport.head(2)

,linea,ramal,stop_name,lat,lon,type
0,Sarmiento,Once - Moreno,Ciudadela,-34.639923,-58.541123,tren
1,Sarmiento,Once - Moreno,Liniers,-34.638873,-58.526341,tren


In [ ]:
df_transport["lat_lon"] = df_transport.apply(lambda x: (x.lat, x.lon), axis=1)
df_transport.head(2)

,linea,ramal,stop_name,lat,lon,type,lat_lon
0,Sarmiento,Once - Moreno,Ciudadela,-34.639923,-58.541123,tren,"(-34.6399227, -58.5411225)"
1,Sarmiento,Once - Moreno,Liniers,-34.638873,-58.526341,tren,"(-34.6388728, -58.5263409)"


In [ ]:
df_transport[df_transport.duplicated(subset=["lat_lon"])]

,linea,ramal,stop_name,lat,lon,type,lat_lon


In [ ]:
unique_lat_lon = df_transport["lat_lon"].unique()
unique_lat_lon[0]

(-34.6399227, -58.5411225)

In [ ]:
def get_closest_transports(lat, lon):
    n_transports = 0
    list_nearest = []
    for lat_lon in unique_lat_lon:
        dist = distance.distance(lat_lon, (lat, lon)).km
        list_nearest.append(dist)
        if dist <= 1:
            n_transports += 1

    closest_transport = min(list_nearest)
    return closest_transport, n_transports

In [ ]:
# df["closest_transport"], df["n_transports"] = zip(*df.swifter.apply(lambda x: get_closest_transports(x.lat, x.lon), axis=1))

In [ ]:
df = df.copy()
# df = df.reset_index(drop=False)

color_scale = [(0, 'orange'), (1,'red')]

fig = None
fig = px.scatter_mapbox(df,
                        lat="lat",
                        lon="lon",
                        hover_name="title",
                        hover_data=["id", "suburb", "published_suburb"],
                        color="province",
                        color_continuous_scale=color_scale,
                        zoom=10,
                        height=800,
                        width=1600)

fig.update_layout(mapbox_style="carto-darkmatter")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
def search_neigborhoods(x, y):
    for i in all_neigborhoods:
        if i.lower() in x.lower():
            return i
        elif i.lower() in y.lower():
            return i
    return np.nan

In [ ]:
def search_price(text):
# for i in df["description"].dropna().sample(100):
    # print(i)
    pattern = r"\b\d{1,7}(?:\.\d{1,7})?\s*(?:usd|dolar|dolares)\b"
    nums = re.findall(pattern, text)
    if nums:
        # print(nums[0] if len(nums)>0 else np.nan)
        # print("\n")
        return float(nums[0])
    else:
        return np.nan